In [1]:
# For colab users
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
cd /content/drive/My Drive/capstone/train_w_freq

/content/drive/My Drive/capstone/train_w_freq


In [3]:
!pip install python-Levenshtein

     |████████████████████████████████| 51kB 5.2MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144807 sha256=dc547163abe1bd3be2887358d7220c087336404fc6ec90bf3f1f483a56d82919
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein


In [4]:
import model as m

import argparse
import os
import time

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
from torch.nn.utils import clip_grad_norm
import torchtext.data as data
import random
import matplotlib.pyplot as plt
from tqdm import notebook


In [5]:
# modified train function
def train(config, train_iter, model, criterion, optimizer, epoch, dic, save_path, prob=0):
    global iteration, n_total, train_loss, n_bad_loss
    global init, best_train_loss, stop

    print("=> EPOCH {}".format(epoch))
    train_iter.init_epoch()
    for i, batch in notebook.tqdm(enumerate(train_iter), total=len(train_iter)):
        # TODO 
        # add noise to batch
        g = batch.grapheme
        # if random.random() > prob:
         #   g = grapheme_add_noise(0.1, g, dic)
        
        
        iteration += 1
        model.train()
        output, _, __ = model(g, batch.phoneme[:-1].detach())
        target = batch.phoneme[1:]
        # print(output.size())
        loss = criterion(output.view(output.size(0) * output.size(1), -1),
                         target.view(target.size(0) * target.size(1)))
        
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), config.clip, 'inf')
        optimizer.step()
        
        n_total += batch.batch_size
        train_loss += loss.data * batch.batch_size
        
        # print("   % Time: {:5.0f} | Iteration: {:5} | Batch: {:4}/{}"
        #           " | Train loss: {:.4f}"
        #           .format(time.time()-init, iteration, train_iter.iterations,
        #                   len(train_iter), train_loss))
        
        if iteration % config.log_every == 0:
            train_loss /= n_total
            print("   % Time: {:5.0f} | Iteration: {:5} | Batch: {:4}/{}"
                  " | Train loss: {:.4f} "
                  .format(time.time()-init, iteration, train_iter.iterations,
                          len(train_iter), train_loss))
            
            
        
            if train_loss < best_train_loss:
                best_train_loss = train_loss
                n_bad_loss = 0
                torch.save(model.state_dict(), save_path)
            else:
                n_bad_loss += 1
            if n_bad_loss == config.n_bad_loss:
                best_train_loss = train_loss
                n_bad_loss = 0
                m.adjust_learning_rate(optimizer, config.lr_decay)
                new_lr = optimizer.param_groups[0]['lr']
                print("=> Adjust learning rate to: {}".format(new_lr))
                if new_lr < config.lr_min:
                    stop = True
                    break
                    
            # test for val_loss improvement
            n_total = train_loss = 0

In [22]:
def test(test_iter, model, g_dic, p_dic, print_error=False):
    model.eval()
    test_iter.init_epoch()
    test_per = 0
    pers = {}
    with torch.no_grad():
        for batch in test_iter:
            output = model(batch.grapheme).data.tolist()
            target = batch.phoneme[1:].squeeze(1).data.tolist()
            # calculate per, wer here
            try:
                per = m.phoneme_error_rate(output[:output.index(3) + 1], target)
            except ValueError:
                per = m.phoneme_error_rate(output, target) 

            


            tmp = list([g_dic[g] for g in batch.grapheme])
            tmp.reverse()
            g = ''.join(tmp[:-1])
            pers[g] = per

            test_per += per  # batch_size = 1
        
        test_per = test_per / len(test_iter.dataset) * 100
        print("Phoneme error rate (PER): {:.2f}\n"
              .format(test_per))
    
    return test_per, pers

In [7]:
def print_result(model, lines, grapheme, g_dic, p_dic):
    g = grapheme[:-3]
    for line in lines:
        if line.strip().split()[0] == g:
            target = line.strip().split()[1:]

    input = [2]
    for letter in g:
        input.insert(1, g_dic.index(letter))
    input = torch.unsqueeze(torch.LongTensor(input).to("cuda"), 1)
    
    with torch.no_grad():

        output = model(input).data.tolist()

     
    print("Grapheme: {}\nTarget: {}\nPrediction: {}\n".format(
            g, ' '.join(target), ' '.join([p_dic[p] for p in output[:-1]]) ) )

In [8]:
parser = {
    'num_row': -1, # modify num_row to -1 if you want to input all data points
    'epochs': 15,
    'batch_size': 128,
    'max_len': 20,  # max length of grapheme/phoneme sequences
    'beam_size': 5,  # size of beam for beam-search
    'd_embed': 30,  # embedding dimension
    'd_hidden': 32,  # hidden dimension
    'attention': True,  # use attention or not
    'log_every': 100,  # number of iterations to log and validate training
    'lr': 0.007,  # initial learning rate
    'lr_decay': 0.5,  # decay lr when not observing improvement in val_loss
    'lr_min': 1e-6,  # stop when lr is too low
    'n_bad_loss': 5,  # number of bad val_loss before decaying
    'clip': 2.3,  # clip gradient, to avoid exploding gradient
    'cuda': True,  # using gpu or not
    'seed': 1234,  # initial seed
}
args = argparse.Namespace(**parser)

In [9]:
args.cuda = args.cuda and torch.cuda.is_available()

# if not os.path.isdir(args.intermediate_path):
#     os.makedirs(args.intermediate_path)
# if not os.path.isdir(args.data_path):
#     os.makedirs(args.data_path)
#     URL = "https://github.com/cmusphinx/cmudict/archive/master.zip"
#     !wget $URL -O ./data/cmudict.zip
#     !unzip ./data/cmudict.zip -d ./data/
#     !mv ./data/cmudict-master $args.data_path

torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

In [10]:
import pandas as pd
import pickle

In [14]:
df = pd.read_csv("../filtered_no_stress_freq.csv", index_col=0)

In [15]:
new_df = df.sample(frac=1, random_state=777)

In [16]:
weight = np.clip(df["frequency"].tolist(), 0, 1000)
np.random.seed(args.seed)
train_lines = np.random.choice(df["word"].tolist(), size=1000000, p=(np.array(weight) / sum(weight)) )

In [17]:
test_lines = []
train_freq = {}
for pair in train_lines:
    tmp = pair.strip().split()
    word = tmp[0]
    try:
        train_freq[word] += 1
    except KeyError:
        test_lines.append(pair)
        train_freq[word] = 1

In [18]:
val_lines = ["this is a sudo line"]

In [19]:
train_iter, val_iter, test_iter, g_field, p_field = m.prepare_data(train_lines, val_lines, test_lines, args)
dic = g_field.vocab.itos

In [20]:
sorted_train_freq = dict(sorted(train_freq.items(), key=lambda item: item[1], reverse=True))

In [21]:
# Training
#set up configuration
config = args
config.g_size = len(g_field.vocab)
config.p_size = len(p_field.vocab)
config.d_hidden = 32
config.epochs = 15
save_path = "../model_para/train_w_freq/train_on_training_" + str(config.d_hidden) + ".pt"
    
model = m.G2P(config)
criterion = nn.NLLLoss()
if config.cuda:
    model.cuda()
    criterion.cuda()
optimizer = optim.Adam(model.parameters(), lr=config.lr)  # use Adagrad
    
# training
iteration = n_total = train_loss = n_bad_loss = 0
stop = False
best_train_loss = 10
init = time.time()

# If you want to change epoch, uncomment the following line
# config.epochs = 10

for epoch in range(1, config.epochs+1):
    train(config, train_iter, model, criterion, optimizer, epoch, dic, save_path)
    if stop:
        break


=> EPOCH 1


   % Time:     6 | Iteration:   100 | Batch:  100/7813 | Train loss: 1.5382 
   % Time:     9 | Iteration:   200 | Batch:  200/7813 | Train loss: 1.0022 
   % Time:    11 | Iteration:   300 | Batch:  300/7813 | Train loss: 0.5687 
   % Time:    13 | Iteration:   400 | Batch:  400/7813 | Train loss: 0.3651 
   % Time:    15 | Iteration:   500 | Batch:  500/7813 | Train loss: 0.2754 
   % Time:    18 | Iteration:   600 | Batch:  600/7813 | Train loss: 0.2268 
   % Time:    20 | Iteration:   700 | Batch:  700/7813 | Train loss: 0.1905 
   % Time:    22 | Iteration:   800 | Batch:  800/7813 | Train loss: 0.1682 
   % Time:    24 | Iteration:   900 | Batch:  900/7813 | Train loss: 0.1547 
   % Time:    26 | Iteration:  1000 | Batch: 1000/7813 | Train loss: 0.1400 
   % Time:    28 | Iteration:  1100 | Batch: 1100/7813 | Train loss: 0.1283 
   % Time:    31 | Iteration:  1200 | Batch: 1200/7813 | Train loss: 0.1193 
   % Time:    33 | Iteration:  1300 | Batch: 1300/7813 | Train loss: 0.1126 

   % Time:   185 | Iteration:  7900 | Batch:   87/7813 | Train loss: 0.0414 
   % Time:   187 | Iteration:  8000 | Batch:  187/7813 | Train loss: 0.0388 
   % Time:   190 | Iteration:  8100 | Batch:  287/7813 | Train loss: 0.0391 
   % Time:   192 | Iteration:  8200 | Batch:  387/7813 | Train loss: 0.0370 
   % Time:   194 | Iteration:  8300 | Batch:  487/7813 | Train loss: 0.0389 
   % Time:   197 | Iteration:  8400 | Batch:  587/7813 | Train loss: 0.0369 
   % Time:   199 | Iteration:  8500 | Batch:  687/7813 | Train loss: 0.0376 
   % Time:   201 | Iteration:  8600 | Batch:  787/7813 | Train loss: 0.0394 
   % Time:   203 | Iteration:  8700 | Batch:  887/7813 | Train loss: 0.0362 
   % Time:   205 | Iteration:  8800 | Batch:  987/7813 | Train loss: 0.0378 
   % Time:   208 | Iteration:  8900 | Batch: 1087/7813 | Train loss: 0.0378 
   % Time:   210 | Iteration:  9000 | Batch: 1187/7813 | Train loss: 0.0389 
   % Time:   212 | Iteration:  9100 | Batch: 1287/7813 | Train loss: 0.0387 

   % Time:   366 | Iteration: 15700 | Batch:   74/7813 | Train loss: 0.0321 
   % Time:   368 | Iteration: 15800 | Batch:  174/7813 | Train loss: 0.0341 
   % Time:   370 | Iteration: 15900 | Batch:  274/7813 | Train loss: 0.0359 
   % Time:   373 | Iteration: 16000 | Batch:  374/7813 | Train loss: 0.0344 
   % Time:   375 | Iteration: 16100 | Batch:  474/7813 | Train loss: 0.0340 
   % Time:   377 | Iteration: 16200 | Batch:  574/7813 | Train loss: 0.0334 
=> Adjust learning rate to: 6.8359375e-06
   % Time:   379 | Iteration: 16300 | Batch:  674/7813 | Train loss: 0.0341 
   % Time:   382 | Iteration: 16400 | Batch:  774/7813 | Train loss: 0.0341 
   % Time:   384 | Iteration: 16500 | Batch:  874/7813 | Train loss: 0.0349 
   % Time:   386 | Iteration: 16600 | Batch:  974/7813 | Train loss: 0.0350 
   % Time:   388 | Iteration: 16700 | Batch: 1074/7813 | Train loss: 0.0343 
=> Adjust learning rate to: 3.41796875e-06
   % Time:   391 | Iteration: 16800 | Batch: 1174/7813 | Train loss:

In [23]:
save_path = "../model_para/train_w_freq/train_on_training_" + str(config.d_hidden) + ".pt"
model = m.G2P(config)
model.load_state_dict(torch.load(save_path))
model.to("cuda")

G2P(
  (encoder): Encoder(
    (embedding): Embedding(32, 30)
    (lstm): LSTMCell(30, 32)
  )
  (decoder): Decoder(
    (embedding): Embedding(47, 30)
    (lstm): LSTMCell(30, 32)
    (attn): Attention(
      (linear): Linear(in_features=64, out_features=32, bias=False)
    )
    (linear): Linear(in_features=32, out_features=47, bias=True)
  )
)

In [24]:
s = time.time()
train_per, pers = test(test_iter, model, g_dic=g_field.vocab.itos, p_dic=p_field.vocab.itos)
print(time.time() - s)

Phoneme error rate (PER): 17.10

231.49782538414001


In [25]:
per_1 = []
per_2_5 = []
per_6_20 = []
per_21_50 = []
per_51_100 = []
per_101_200 = []
per_200_p = []
for key, value in enumerate(sorted_train_freq):
    if pers[value] > 1:
        continue;

    if sorted_train_freq[value] == 1:
        per_1.append(pers[value])
    elif sorted_train_freq[value] <= 5:
        per_2_5.append(pers[value])
    elif sorted_train_freq[value] <= 20:
        per_6_20.append(pers[value])
    elif sorted_train_freq[value] <= 50:
        per_21_50.append(pers[value])
    elif sorted_train_freq[value] <= 100:
        per_51_100.append(pers[value])
    elif sorted_train_freq[value] <= 200:
        per_101_200.append(pers[value])
    else:
        per_200_p.append(pers[value]) 

In [26]:
np.mean(per_1) * 100, np.mean(per_2_5) * 100, np.mean(per_6_20) * 100, np.mean(per_21_50) * 100, np.mean(per_51_100) * 100, np.mean(per_101_200) * 100, np.mean(per_200_p) * 100

(19.52442952813598,
 18.8820109632979,
 16.438720399638278,
 13.885973920290509,
 12.221644028105201,
 12.412102292435279,
 14.955156084376103)

In [27]:
len(per_1), len(per_2_5), len(per_6_20), len(per_21_50), len(per_51_100), len(per_101_200), len(per_200_p), 

(4047, 11573, 9322, 3424, 1687, 961, 859)

In [28]:
per_1 = []
per_2 = []
per_3 = []
per_4 = []
per_5_10 = []
per_11_20 = []
per_20_p = []
for key, value in enumerate(sorted_train_freq):
    if pers[value] > 1:
        continue;

    if sorted_train_freq[value] == 1:
        per_1.append(pers[value])
    elif sorted_train_freq[value] == 2:
        per_2.append(pers[value])
    elif sorted_train_freq[value] == 3:
        per_3.append(pers[value])
    elif sorted_train_freq[value] == 4:
        per_4.append(pers[value])
    elif sorted_train_freq[value] <= 10:
        per_5_10.append(pers[value])
    elif sorted_train_freq[value] <= 20:
        per_11_20.append(pers[value])
    else:
        per_20_p.append(pers[value]) 

In [29]:
np.mean(per_1) * 100, np.mean(per_2) * 100, np.mean(per_3) * 100, np.mean(per_4) * 100, np.mean(per_5_10) * 100, np.mean(per_11_20) * 100, np.mean(per_20_p) * 100

(19.52442952813598,
 19.577900165683833,
 19.16628755334619,
 17.89626293191625,
 17.43312703020421,
 15.540580632111695,
 13.40903153340031)

In [30]:
len(per_1), len(per_2), len(per_3), len(per_4), len(per_5_10), len(per_11_20), len(per_20_p), 

(4047, 3976, 3172, 2481, 7210, 4056, 6931)